<a href="https://colab.research.google.com/github/prasanth9989/assignment/blob/main/pspyk12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#user_data
%%writefile user_data.csv
user_id,series,season,timestamp,genre,duration_mins
521,”Mirzapur”,3,2024-07-30 15:00:00,action,300
672,”Panchayat”,3,2024-07-30 15:00:00,comedy,200
197,”Family Man”,2,2024-07-30 15:00:00,action,500
521,”Mirzapur”,2,2024-07-29 15:00:00,action,280
211,”Queens Gambit”,1,2024-07-30 15:00:00,drama,170
521,”Mirzapur”,1,2024-07-28 15:00:00,action,230
844,”Westworld”,3,2024-07-30 15:00:00,sci-fi,310
672,”Panchayat”,3,2024-07-29 15:00:00,comedy,210
256,”Homecoming”,2,2024-07-30 15:00:00,thriller,310
489,”Outer Range”,1,2024-07-30 15:00:00,sci-fi,340
200,”Black Mirror”,2,2024-07-30 15:00:00,sci-fi,140
256,”Outer Range”,2,2024-07-30 15:00:00,thriller,250
489,”Outer Range”,2,2024-07-28 15:00:00,sci-fi,170
200,”Black Mirror”,3,2024-07-29 15:00:00,sci-fi,190
672,”Panchayat”,2,2024-07-28 15:00:00,comedy,160
672,”Outer Range”,1,2024-07-25 15:00:00,sci-fi,250
200,”Black Mirror”,4,2024-07-28 15:00:00,sci-fi,200
844,”Westworld”,2,2024-07-29 15:00:00,sci-fi,300
672,”Black Mirror”,5,2024-07-28 15:00:00,sci-fi,150
672,”Panchayat”,1,2024-07-27 15:00:00,comedy,190

Writing user_data.csv


In [2]:
#Import Libraries
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=fb4c96fd49c1fab30340a56a86cd093d700616898c1c7cb0d68234cd134c74c7
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
#Create SparkSession for app (Streaming Analysis)
spark=SparkSession.builder.appName('streaming Analysis').getOrCreate()

In [5]:
#Create Dataframe series_df
df=spark.read.option('header','True').csv('user_data.csv')
df.show()

+-------+---------------+------+-------------------+--------+-------------+
|user_id|         series|season|          timestamp|   genre|duration_mins|
+-------+---------------+------+-------------------+--------+-------------+
|    521|     ”Mirzapur”|     3|2024-07-30 15:00:00|  action|          300|
|    672|    ”Panchayat”|     3|2024-07-30 15:00:00|  comedy|          200|
|    197|   ”Family Man”|     2|2024-07-30 15:00:00|  action|          500|
|    521|     ”Mirzapur”|     2|2024-07-29 15:00:00|  action|          280|
|    211|”Queens Gambit”|     1|2024-07-30 15:00:00|   drama|          170|
|    521|     ”Mirzapur”|     1|2024-07-28 15:00:00|  action|          230|
|    844|    ”Westworld”|     3|2024-07-30 15:00:00|  sci-fi|          310|
|    672|    ”Panchayat”|     3|2024-07-29 15:00:00|  comedy|          210|
|    256|   ”Homecoming”|     2|2024-07-30 15:00:00|thriller|          310|
|    489|  ”Outer Range”|     1|2024-07-30 15:00:00|  sci-fi|          340|
|    200| ”B

In [6]:
a=df.select('series').distinct().show()

+---------------+
|         series|
+---------------+
|   ”Family Man”|
|   ”Homecoming”|
|    ”Panchayat”|
|  ”Outer Range”|
| ”Black Mirror”|
|     ”Mirzapur”|
|    ”Westworld”|
|”Queens Gambit”|
+---------------+



In [7]:
#Find the user with maximum watchtime
arr1=df.groupBy('user_id').agg({'duration_mins':'sum'}).orderBy('sum(duration_mins)',ascending=False).show(1)

+-------+------------------+
|user_id|sum(duration_mins)|
+-------+------------------+
|    672|            1160.0|
+-------+------------------+
only showing top 1 row



In [8]:
#Calculate overall total Watchtime
arr2=df.groupBy('user_id').agg({'duration_mins':'sum'}).show()

+-------+------------------+
|user_id|sum(duration_mins)|
+-------+------------------+
|    521|             810.0|
|    200|             530.0|
|    672|            1160.0|
|    256|             560.0|
|    197|             500.0|
|    211|             170.0|
|    844|             610.0|
|    489|             510.0|
+-------+------------------+



In [9]:
#Find most popular shows (based on watchtime)
arr3=df.groupBy('series','genre').agg({'duration_mins':'sum'}).orderBy('sum(duration_mins)',ascending=False).show()

+---------------+--------+------------------+
|         series|   genre|sum(duration_mins)|
+---------------+--------+------------------+
|     ”Mirzapur”|  action|             810.0|
|  ”Outer Range”|  sci-fi|             760.0|
|    ”Panchayat”|  comedy|             760.0|
| ”Black Mirror”|  sci-fi|             680.0|
|    ”Westworld”|  sci-fi|             610.0|
|   ”Family Man”|  action|             500.0|
|   ”Homecoming”|thriller|             310.0|
|  ”Outer Range”|thriller|             250.0|
|”Queens Gambit”|   drama|             170.0|
+---------------+--------+------------------+



In [10]:
#Find most popular shows (based on user popularity)
arr4=df.groupBy('series').agg({'user_id':'count'}).orderBy('count(user_id)',ascending=False).show()

+---------------+--------------+
|         series|count(user_id)|
+---------------+--------------+
|    ”Panchayat”|             4|
|  ”Outer Range”|             4|
| ”Black Mirror”|             4|
|     ”Mirzapur”|             3|
|    ”Westworld”|             2|
|   ”Family Man”|             1|
|   ”Homecoming”|             1|
|”Queens Gambit”|             1|
+---------------+--------------+



In [11]:
#Find the most popular genre
arr5=df.groupBy('genre').agg({'genre':'count'}).orderBy('count(genre)',ascending=False).show(1)

+------+------------+
| genre|count(genre)|
+------+------------+
|sci-fi|           9|
+------+------------+
only showing top 1 row



In [12]:
#Calculate total watchtime per user
arr6=df.groupBy('user_id').agg({'duration_mins':'sum'}).show()

+-------+------------------+
|user_id|sum(duration_mins)|
+-------+------------------+
|    521|             810.0|
|    200|             530.0|
|    672|            1160.0|
|    256|             560.0|
|    197|             500.0|
|    211|             170.0|
|    844|             610.0|
|    489|             510.0|
+-------+------------------+



In [13]:
df.show()

+-------+---------------+------+-------------------+--------+-------------+
|user_id|         series|season|          timestamp|   genre|duration_mins|
+-------+---------------+------+-------------------+--------+-------------+
|    521|     ”Mirzapur”|     3|2024-07-30 15:00:00|  action|          300|
|    672|    ”Panchayat”|     3|2024-07-30 15:00:00|  comedy|          200|
|    197|   ”Family Man”|     2|2024-07-30 15:00:00|  action|          500|
|    521|     ”Mirzapur”|     2|2024-07-29 15:00:00|  action|          280|
|    211|”Queens Gambit”|     1|2024-07-30 15:00:00|   drama|          170|
|    521|     ”Mirzapur”|     1|2024-07-28 15:00:00|  action|          230|
|    844|    ”Westworld”|     3|2024-07-30 15:00:00|  sci-fi|          310|
|    672|    ”Panchayat”|     3|2024-07-29 15:00:00|  comedy|          210|
|    256|   ”Homecoming”|     2|2024-07-30 15:00:00|thriller|          310|
|    489|  ”Outer Range”|     1|2024-07-30 15:00:00|  sci-fi|          340|
|    200| ”B

In [14]:
#Find most popular genre (based on engagement count)
arr7=df.groupBy('genre').agg({'duration_mins':'count'}).show()

+--------+--------------------+
|   genre|count(duration_mins)|
+--------+--------------------+
|  action|                   4|
|   drama|                   1|
|thriller|                   2|
|  sci-fi|                   9|
|  comedy|                   4|
+--------+--------------------+



In [15]:
arr8=df.groupBy('genre').agg({'genre':'count'}).show()

+--------+------------+
|   genre|count(genre)|
+--------+------------+
|  action|           4|
|   drama|           1|
|thriller|           2|
|  sci-fi|           9|
|  comedy|           4|
+--------+------------+



In [16]:
#Find average watchtime per genre
arr9=df.groupBy('genre').agg({'duration_mins':'avg'}).show()

+--------+------------------+
|   genre|avg(duration_mins)|
+--------+------------------+
|  action|             327.5|
|   drama|             170.0|
|thriller|             280.0|
|  sci-fi|227.77777777777777|
|  comedy|             190.0|
+--------+------------------+



In [17]:
df.show()

+-------+---------------+------+-------------------+--------+-------------+
|user_id|         series|season|          timestamp|   genre|duration_mins|
+-------+---------------+------+-------------------+--------+-------------+
|    521|     ”Mirzapur”|     3|2024-07-30 15:00:00|  action|          300|
|    672|    ”Panchayat”|     3|2024-07-30 15:00:00|  comedy|          200|
|    197|   ”Family Man”|     2|2024-07-30 15:00:00|  action|          500|
|    521|     ”Mirzapur”|     2|2024-07-29 15:00:00|  action|          280|
|    211|”Queens Gambit”|     1|2024-07-30 15:00:00|   drama|          170|
|    521|     ”Mirzapur”|     1|2024-07-28 15:00:00|  action|          230|
|    844|    ”Westworld”|     3|2024-07-30 15:00:00|  sci-fi|          310|
|    672|    ”Panchayat”|     3|2024-07-29 15:00:00|  comedy|          210|
|    256|   ”Homecoming”|     2|2024-07-30 15:00:00|thriller|          310|
|    489|  ”Outer Range”|     1|2024-07-30 15:00:00|  sci-fi|          340|
|    200| ”B

In [18]:
#Find peak traffic days
#(Output 1 = Full Date)
arr10=df.groupBy(to_date('timestamp')).agg(count('timestamp')).orderBy('count(timestamp)',ascending=False)
arr10.select('to_date(timestamp)').show(1)


+------------------+
|to_date(timestamp)|
+------------------+
|        2024-07-30|
+------------------+
only showing top 1 row



In [19]:
#(Output 2 = Only Day)
arr10=df.groupBy(dayofmonth('timestamp')).agg(count('timestamp')).orderBy('count(timestamp)',ascending=False)
arr10.select('dayofmonth(timestamp)').show(1)


+---------------------+
|dayofmonth(timestamp)|
+---------------------+
|                   30|
+---------------------+
only showing top 1 row



In [20]:
df.show()

+-------+---------------+------+-------------------+--------+-------------+
|user_id|         series|season|          timestamp|   genre|duration_mins|
+-------+---------------+------+-------------------+--------+-------------+
|    521|     ”Mirzapur”|     3|2024-07-30 15:00:00|  action|          300|
|    672|    ”Panchayat”|     3|2024-07-30 15:00:00|  comedy|          200|
|    197|   ”Family Man”|     2|2024-07-30 15:00:00|  action|          500|
|    521|     ”Mirzapur”|     2|2024-07-29 15:00:00|  action|          280|
|    211|”Queens Gambit”|     1|2024-07-30 15:00:00|   drama|          170|
|    521|     ”Mirzapur”|     1|2024-07-28 15:00:00|  action|          230|
|    844|    ”Westworld”|     3|2024-07-30 15:00:00|  sci-fi|          310|
|    672|    ”Panchayat”|     3|2024-07-29 15:00:00|  comedy|          210|
|    256|   ”Homecoming”|     2|2024-07-30 15:00:00|thriller|          310|
|    489|  ”Outer Range”|     1|2024-07-30 15:00:00|  sci-fi|          340|
|    200| ”B

In [24]:
#Find the user with most diverse show preference
df.groupBy('user_id').agg({'series':'count'}).orderBy('count(series)',ascending=False).limit(1).collect()[0][0]


'672'

In [28]:
#Find the binge-watchers
df1=df.groupBy('user_id','timestamp','series').agg({'season':'count'}).withColumnRenamed('count(season)','count').orderBy('user_id')
df2=df.groupBy('series').agg({'season':'max'}).withColumnRenamed('max(season)','max')
df3=df1.join(df2,on='series',how='inner')
df3.show()
df3.filter(col('count')==col('max')).collect()[0][1]

+---------------+-------+-------------------+-----+---+
|         series|user_id|          timestamp|count|max|
+---------------+-------+-------------------+-----+---+
|     ”Mirzapur”|    521|2024-07-30 15:00:00|    1|  3|
|    ”Westworld”|    844|2024-07-30 15:00:00|    1|  3|
| ”Black Mirror”|    672|2024-07-28 15:00:00|    1|  5|
|   ”Homecoming”|    256|2024-07-30 15:00:00|    1|  2|
| ”Black Mirror”|    200|2024-07-29 15:00:00|    1|  5|
|  ”Outer Range”|    672|2024-07-25 15:00:00|    1|  2|
|  ”Outer Range”|    489|2024-07-28 15:00:00|    1|  2|
|  ”Outer Range”|    256|2024-07-30 15:00:00|    1|  2|
|    ”Panchayat”|    672|2024-07-27 15:00:00|    1|  3|
|  ”Outer Range”|    489|2024-07-30 15:00:00|    1|  2|
|    ”Panchayat”|    672|2024-07-29 15:00:00|    1|  3|
|     ”Mirzapur”|    521|2024-07-28 15:00:00|    1|  3|
|”Queens Gambit”|    211|2024-07-30 15:00:00|    1|  1|
| ”Black Mirror”|    200|2024-07-28 15:00:00|    1|  5|
|    ”Panchayat”|    672|2024-07-30 15:00:00|   

'211'

In [25]:
#Find the user with longest watching streak
df.groupBy('user_id').agg({'timestamp':'count'}).orderBy('count(timestamp)',ascending=False).show(1)


+-------+----------------+
|user_id|count(timestamp)|
+-------+----------------+
|    672|               6|
+-------+----------------+
only showing top 1 row



In [22]:
#Total Seasons available
arr12=df.groupBy('series').agg(max('season')).orderBy('max(season)',ascending=False).show()

+---------------+-----------+
|         series|max(season)|
+---------------+-----------+
| ”Black Mirror”|          5|
|     ”Mirzapur”|          3|
|    ”Panchayat”|          3|
|    ”Westworld”|          3|
|   ”Family Man”|          2|
|   ”Homecoming”|          2|
|  ”Outer Range”|          2|
|”Queens Gambit”|          1|
+---------------+-----------+



In [23]:
#Fetch a list of all series
arr13=df.select('series').distinct()
arr13.rdd.flatMap(lambda x:[x[0]]).collect()

['”Family Man”',
 '”Homecoming”',
 '”Panchayat”',
 '”Outer Range”',
 '”Black Mirror”',
 '”Mirzapur”',
 '”Westworld”',
 '”Queens Gambit”']